# 1. 데이터 불러오기

In [2]:
max_len = 256 # maximum size of lyrics

In [1]:
import pandas as pd

In [3]:
df = pd.read_csv('sms.tsv', sep='\t',)
print(df.columns, df.shape)

(Index([u'label', u'sms'], dtype='object'), (5572, 2))


In [4]:
df.head()

,label,sms
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


## 데이터 전처리

In [5]:
# 클래스 파악
classes = sorted(set(df['label']))
class_to_idx = {}

for i, c in enumerate(classes): # about all classes
    class_to_idx.update({c:i})

nclass = len(classes)
print("# of classes : %d" %nclass)
print(classes)
print(class_to_idx)
    

# of classes : 2
['ham', 'spam']
{'ham': 0, 'spam': 1}


# 2. 새로운 DataFrame

## 1) 'label, sms'만 남기기
## 2) 최대 텍스트 길이 만큼 자르기 # pandas.Series.str.slice
- '성별, 가사'만 남기려면?

In [6]:
new_df = pd.DataFrame({'label' :df['label'], 'sms':df['sms'].str.slice(# 최대 가사 텍스트만큼 자르기
                        start = 0, stop = max_len)})

# 3) 중복 제거

In [7]:
len(new_df)

5572

In [15]:
new_df = pd.DataFrame(new_df.drop_duplicates())

In [16]:
len(new_df)

5169

# 4) 셔플

In [17]:
df_shuffled = new_df.sample(frac=1).reset_index(drop=True)
df_shuffled.head()

,label,sms
0,ham,Gam gone after outstanding innings.
1,ham,Tomarrow final hearing on my laptop case so i ...
2,spam,"SIX chances to win CASH! From 100 to 20,000 po..."
3,ham,Do u noe how 2 send files between 2 computers?
4,ham,"Dear, take care. I am just reaching home.love ..."


# 5) train, test 나누기

In [18]:
# train : test = 9 : 1
# train : test = 540 : 6 -> train:valid:test = 432:108:60
train_ratio = 0.9

# train dataset
s, e = 0, int(df_shuffled.shape[0] * train_ratio) # # of rows
df_train = pd.DataFrame({'label' :df_shuffled['label'][s:e], 'sms':df_shuffled['sms'][s:e]})

print("index for train : %d ~ %d" %(s,e))

# test dataset
s, e = e, e + int(df_shuffled.shape[0] * (1.0 - train_ratio))
print("index for test : %d ~ %d" %(s,e))
df_test = pd.DataFrame({'label' :df_shuffled['label'][s:e], 'sms':df_shuffled['sms'][s:e]})


index for train : 0 ~ 4652
index for test : 4652 ~ 5168


In [19]:
# column 수 확인
print(df_train.shape)
print(df_test.shape)

(4652, 2)
(516, 2)


# 6) 저장

In [20]:
# new_df.columns : ['성별', '가사']
df_train.to_csv('./sms.maxlen.uniq.shuf.train.tsv', header = False, index = False, sep = '\t')
df_test.to_csv('./sms.maxlen.uniq.shuf.test.tsv', header = False, index = False, sep = '\t')